# Basics: how NMEA Parser work

All NMEA Strings are made out of several parts:
- A '$' sign
- A Device (aka talker) ID
- A Sentence ID
- The Sentence data
- A Checksum, preceded with a star
- A String terminator, usually CR-LF (Carriage Return, and Line Feed)

Example, RMC String:
```
 $GPRMC,123519,A,4807.038,N,01131.000,E,022.4,084.4,230394,003.1,W,T*6A
 || |   |                                                            |
 || |   |                                                            Checksum, 6A
 || |   Data, comma-separated, 123519,...,T
 || Sentence ID, RMC
 |Talker ID, GP
 Dollar sign
```


The Data, per the NMEA Spec:

```
                                                                  12
       1      2 3        4 5         6 7     8     9      10    11
$GPRMC,123519,A,4807.038,N,01131.000,E,022.4,084.4,230394,003.1,W,T*6A
       |      | |        | |         | |     |     |      |     | |
       |      | |        | |         | |     |     |      |     | Type
       |      | |        | |         | |     |     |      |     Variation sign
       |      | |        | |         | |     |     |      Variation value
       |      | |        | |         | |     |     Date DDMMYY
       |      | |        | |         | |     COG
       |      | |        | |         | SOG
       |      | |        | |         Longitude Sign
       |      | |        | Longitude Value
       |      | |        Latitude Sign
       |      | Latitude value
       |      Active or Void
       UTC

```
The Sentence IDs, and their corresponding content and structure are defined by the NMEA specification.

## The checksum

The first thing to do is to validate the checksum; this will tell you if the sentence is valid, and is worth parsing.

The checksum validation concerns the part of the string _after_ the `$` sign, and _before_ the `*`.  
In the above, this would be `GPRMC,123519,A,4807.038,N,01131.000,E,022.4,084.4,230394,003.1,W,T`.

The checksum is the HexaDecimal value of the XOR (aka eXclusive OR) value of each byte of the string to validate.

Here is a Java example of such an operation:

In [1]:
String nmeaSentence = "$GPRMC,123519,A,4807.038,N,01131.000,E,022.4,084.4,230394,003.1,W*6A\r\n"; // No Type here
String str = nmeaSentence.substring(1, nmeaSentence.indexOf("*")); // Assume there IS a star
System.out.printf("Validating %s\n", str);

int cs = 0;
char[] ca = str.toCharArray();
cs = ca[0];
for (int i = 1; i < ca.length; i++) {
  cs = cs ^ ca[i]; // XOR
  System.out.printf("...Checksum is now 0x%02X \n", cs);
}
System.out.printf("Final Checksum 0x%02X (decimal %d)\n", cs, cs);

Validating GPRMC,123519,A,4807.038,N,01131.000,E,022.4,084.4,230394,003.1,W
...Checksum is now 0x17 
...Checksum is now 0x45 
...Checksum is now 0x08 
...Checksum is now 0x4B 
...Checksum is now 0x67 
...Checksum is now 0x56 
...Checksum is now 0x64 
...Checksum is now 0x57 
...Checksum is now 0x62 
...Checksum is now 0x53 
...Checksum is now 0x6A 
...Checksum is now 0x46 
...Checksum is now 0x07 
...Checksum is now 0x2B 
...Checksum is now 0x1F 
...Checksum is now 0x27 
...Checksum is now 0x17 
...Checksum is now 0x20 
...Checksum is now 0x0E 
...Checksum is now 0x3E 
...Checksum is now 0x0D 
...Checksum is now 0x35 
...Checksum is now 0x19 
...Checksum is now 0x57 
...Checksum is now 0x7B 
...Checksum is now 0x4B 
...Checksum is now 0x7A 
...Checksum is now 0x4B 
...Checksum is now 0x78 
...Checksum is now 0x49 
...Checksum is now 0x67 
...Checksum is now 0x57 
...Checksum is now 0x67 
...Checksum is now 0x57 
...Checksum is now 0x7B 
...Checksum is now 0x3E 
...Checksum is now 0x12 

java.io.PrintStream@5ad67260

If the calculated checksum is the same as the one provided in the string itself, then the string is valid. We are looking for a `6A` here.

The string happens here to be valid, we can proceed to parsing.

Let's split the data into an array of elements:

In [2]:
String[] data = str.split(",");

In [3]:
Arrays.asList(data).forEach(System.out::println);

GPRMC
123519
A
4807.038
N
01131.000
E
022.4
084.4
230394
003.1
W


Then, the NMEA spec tells us what element corresponds to what data (starting with index 1).  Look [here](https://gpsd.gitlab.io/gpsd/NMEA.html#_rmc_recommended_minimum_navigation_information) for RMC structure.

**_Important Warning_**: The data are comma-separated, and for the numbers, the decimal separator is `.`. When parsing the data, do make sure the `Locale` is set correctly. For example, if the `Locale` is set to `fr_FR`, the decimal separator will be `,`... which can indeed lead to unexpected results.

In [4]:
System.out.printf("User Locale: %s_%s\n", System.getProperty("user.language"), System.getProperty("user.country"));

User Locale: en_US


java.io.PrintStream@5ad67260

Good luck!